In [1]:
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
import csv

--------------
## Problems that were discovered later on:
we had to come back here to take care of them

### <font color='red'>Problem1:</font> [Incomplete data](#Incomplete_Data)

--------------

In [2]:
# Our base url
URL = "https://www.metacritic.com"

We have seen that some of the data in the links are not accurate for computer users

But cell phone users did not have this problem

### So we defined two user agents:
- 'HEADERS_MOBILE' - cell phone user agent

- 'HEADERS' - computer user agent

In [3]:
# user-agents with every request
HEADERS = {'User-Agent': 'Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148'}
HEADERS_MOBILE = { 'User-Agent' : 'Mozilla/5.0 (iPhone; CPU iPhone OS 9_1 like Mac OS X) AppleWebKit/601.1.46 (KHTML, like Gecko) Version/9.0 Mobile/13B137 Safari/601.1'}

## Helper functions for the 'get_games_data' function:

In [4]:
# Creating the dictionary storing the information of each game
def games_attributes_init():
    
    data = {
        'game_name': [],
        'platform': [],
        'publisher': [],
        'release_date': [],
        'meta_scroe': [],
        'user_score': [],
        'develeoper': [],
        'genres': [],
        'num_of_players': [],
        'rating': [],
        'user_positive_review': [],
        'user_negative_review': [],
        'user_mixed_review': [],
        'critic_positive_review': [],
        'critic_negative_review': [],
        'critic_mixed_review': [],
    }
    
    return data

In [5]:
# Saves or updates the csv file
# ** if ther is index_list: append to csv with index_list, else: make new csv ** #
def save_data_to_csv_file(data, name, index_list=None):
    if(index_list):
        df = pd.DataFrame(data=data, index=index_list)
        df.to_csv(f'games_data/{name}.csv', mode='a', header=False, index=index_list)
    else:
        df = pd.DataFrame(data=data)
        df.to_csv(f'games_data/{name}.csv')

In [6]:
# Checks the status code of the request results
def check_status_code(result, mobile_result):
    if((result.status_code != 200) or (mobile_result.status_code != 200)):
        print(f"Status code: {result.status_code}")
        print(f"Status code mobile: {mobile_result.status_code}")
        return False
    return True
    

# The main function To store game data from the site

In [7]:
def get_games_data(href_csv_file, save_name):

    games_attributes = games_attributes_init()
    starting_index = 1

    try:
        with open(f'games_data/{save_name}.csv', 'r') as game_file:
            games_list = list(csv.reader(game_file))
            last_row = games_list[-1][0]
            if(last_row):
                starting_index = int(last_row) + 2
    except:
        save_data_to_csv_file(games_attributes, save_name)
        
    with open(href_csv_file, 'r') as csv_file:
        hrefs = list(csv.reader(csv_file))
    
    
    indexes = []
    for href in hrefs[starting_index:]:
        print(f"Processing: {href[0]}")
        
        url = URL + href[1]
        result = requests.get(url, headers=HEADERS)
        mobile_result = requests.get(url, headers=HEADERS_MOBILE)
        
        if(check_status_code(result, mobile_result) is False):
            continue
        
        indexes.append(int(href[0]))
        
        mobile_soup = BeautifulSoup(mobile_result.content, "html.parser")
        soup = BeautifulSoup(result.content, "html.parser")
        
        game_name_soup = soup.find("div", class_="product_title")
        games_attributes['game_name'].append(game_name_soup.h1.text.strip())
        
        platform_soup = soup.find("div", class_="product_title").span
        games_attributes['platform'].append(platform_soup.text.strip())
        
        release_date_soup = soup.find("li", class_="summary_detail release_data").find("span", class_="data")
        games_attributes['release_date'].append(release_date_soup.text.strip())
        
        user_score_soup = soup.find("div", class_="userscore_wrap")
        if(user_score_soup):
            games_attributes['user_score'].append(user_score_soup.find(class_="metascore_w").text.strip())
        else:
            games_attributes['user_score'].append("NaN")
        
        meta_scroe_soup = soup.find("span", itemprop="ratingValue")
        if(meta_scroe_soup):
            games_attributes['meta_scroe'].append(meta_scroe_soup.text.strip())
        else:
            games_attributes['meta_scroe'].append("NaN")
        
        publisher_soup = soup.find("li", class_="summary_detail publisher")
        if(publisher_soup):
            games_attributes['publisher'].append(publisher_soup.a.text.strip())
        else:
            games_attributes['publisher'].append("NaN")
        
        develeoper_soup = soup.find("li", class_="summary_detail developer")
        if(develeoper_soup):
            games_attributes['develeoper'].append(develeoper_soup.a.text.strip())
        else:
            games_attributes['develeoper'].append("NaN")

        num_of_players_soup = soup.find("li", class_="summary_detail product_players")
        if(num_of_players_soup):
            games_attributes['num_of_players'].append(num_of_players_soup.find(class_="data").text.strip())
        else:
            games_attributes['num_of_players'].append("NaN")
        
        rating_soup = soup.find("li", class_="summary_detail product_rating")
        if(rating_soup):
            games_attributes['rating'].append(rating_soup.find(class_= "data").text.strip())
        else:
            games_attributes['rating'].append("NaN")

        genres_soup = soup.find("li", class_="summary_detail product_genre").find_all("span")
        game_genres = []
        for genre in genres_soup[1:]:
            game_genres.append(genre.text.strip())
        geners_string = ", ".join(game_genres)
        games_attributes['genres'].append(geners_string)

        critic_user_distributations = mobile_soup.find_all("div", class_= "distributions")
        # ***** critic review Distributations **** #
        if(len(critic_user_distributations) > 0):
            critic_distributions = critic_user_distributations[0].find_all(class_="number")
            games_attributes['critic_positive_review'].append(critic_distributions[0].text.split()[0])
            games_attributes['critic_negative_review'].append(critic_distributions[2].text.split()[0])
            games_attributes['critic_mixed_review'].append(critic_distributions[1].text.split()[0])
        else:
            games_attributes['critic_positive_review'].append("NaN")
            games_attributes['critic_negative_review'].append("NaN")
            games_attributes['critic_mixed_review'].append("NaN")
        
        # ***** User review Distributations **** #
        if(len(critic_user_distributations) > 1):
            user_distributions = critic_user_distributations[1].find_all(class_="number")
            games_attributes['user_positive_review'].append(user_distributions[0].text.split()[0])
            games_attributes['user_negative_review'].append(user_distributions[2].text.split()[0])
            games_attributes['user_mixed_review'].append(user_distributions[1].text.split()[0])
        else:
            games_attributes['user_positive_review'].append("NaN")
            games_attributes['user_negative_review'].append("NaN")
            games_attributes['user_mixed_review'].append("NaN")
        
        # ***** saving changes evry 50 loops **** #
        num_of_games = int(href[0])
        if((num_of_games % 50 == 0) and (num_of_games != 0)):
            save_data_to_csv_file(games_attributes, save_name, index_list=indexes)
            print("saved")
            #  reseting games_attributes and indexes
            games_attributes = games_attributes_init()
            indexes = []


    if(len(indexes) > 0):
        save_data_to_csv_file(games_attributes, save_name, index_list=indexes)
    print("fin")


## saving the Top Rated Games, by runing the function to the 'good_games.csv' file

In [7]:
get_games_data('hrefs_csv/good_games.csv', 'top_rated_games')

Processing: 3501
Processing: 3502
Processing: 3503
Processing: 3504
Processing: 3505
Processing: 3506
Processing: 3507
Processing: 3508
Processing: 3509
Processing: 3510
Processing: 3511
Processing: 3512
Processing: 3513
Processing: 3514
Processing: 3515
Processing: 3516
Processing: 3517
Processing: 3518
Processing: 3519
Processing: 3520
Processing: 3521
Processing: 3522
Processing: 3523
Processing: 3524
Processing: 3525
Processing: 3526
Processing: 3527
Processing: 3528
Processing: 3529
Processing: 3530
Processing: 3531
Processing: 3532
Processing: 3533
Processing: 3534
Processing: 3535
Processing: 3536
Processing: 3537
Processing: 3538
Processing: 3539
Processing: 3540
Processing: 3541
Processing: 3542
Processing: 3543
Processing: 3544
Processing: 3545
Processing: 3546
Processing: 3547
Processing: 3548
Processing: 3549
Processing: 3550
saved
Processing: 3551
Processing: 3552
Processing: 3553
Processing: 3554
Processing: 3555
Processing: 3556
Processing: 3557
Processing: 3558
Processi

After a problem in some parts of the site
We made a slight change in the function and re-run it

In [9]:
get_games_data('hrefs_csv/good_games.csv', 'top_rated_games')

Processing: 3751
Processing: 3752
Processing: 3753
Processing: 3754
Processing: 3755
Processing: 3756
Processing: 3757
Processing: 3758
Processing: 3759
Processing: 3760
Processing: 3761
Processing: 3762
Processing: 3763
Processing: 3764
Processing: 3765
Processing: 3766
Processing: 3767
Processing: 3768
Processing: 3769
Processing: 3770
Processing: 3771
Processing: 3772
Processing: 3773
Processing: 3774
Processing: 3775
Processing: 3776
Processing: 3777
Processing: 3778
Processing: 3779
Processing: 3780
Processing: 3781
Processing: 3782
Processing: 3783
Processing: 3784
Processing: 3785
Processing: 3786
Processing: 3787
Processing: 3788
Processing: 3789
Processing: 3790
Processing: 3791
Processing: 3792
Processing: 3793
Processing: 3794
Processing: 3795
Status code: 404
Status code mobile: 404
Processing: 3796
Processing: 3797
Processing: 3798
Processing: 3799
Processing: 3800
saved
Processing: 3801
Processing: 3802
Processing: 3803
Processing: 3804
Processing: 3805
Processing: 3806
P

## saving the Lowest Rated Games, by runing the function to the 'bad_games.csv' file

In [36]:
get_games_data('hrefs_csv/bad_games.csv', 'lowest_rated_games')

Processing: 1551
Processing: 1552
Processing: 1553
Processing: 1554
Processing: 1555
Processing: 1556
Processing: 1557
Processing: 1558
Processing: 1559
Processing: 1560
Processing: 1561
Processing: 1562
Processing: 1563
Processing: 1564
Processing: 1565
Processing: 1566
Processing: 1567
Processing: 1568
Processing: 1569
Processing: 1570
Processing: 1571
Processing: 1572
Processing: 1573
Processing: 1574
Processing: 1575
Processing: 1576
Processing: 1577
Processing: 1578
Processing: 1579
Processing: 1580
Processing: 1581
Processing: 1582
Processing: 1583
Processing: 1584
Processing: 1585
Processing: 1586
Processing: 1587
Processing: 1588
Processing: 1589
Processing: 1590
Processing: 1591
Processing: 1592
Processing: 1593
Processing: 1594
Processing: 1595
Processing: 1596
Processing: 1597
Processing: 1598
Processing: 1599
Processing: 1600
saved
Processing: 1601
Processing: 1602
Processing: 1603
Processing: 1604
Processing: 1605
Processing: 1606
Processing: 1607
Processing: 1608
Processi

AttributeError: 'NoneType' object has no attribute 'find'

After a problem in some parts of the site
We made a slight change in the function and re-run it

In [42]:
get_games_data('hrefs_csv/bad_games.csv', 'lowest_rated_games')

Processing: 1851
Processing: 1852
Processing: 1853
Processing: 1854
Processing: 1855
Processing: 1856
Processing: 1857
Processing: 1858
Processing: 1859
Processing: 1860
Processing: 1861
Processing: 1862
Processing: 1863
Processing: 1864
Processing: 1865
Processing: 1866
Processing: 1867
Processing: 1868
Processing: 1869
Processing: 1870
Processing: 1871
Processing: 1872
Processing: 1873
Processing: 1874
Processing: 1875
Processing: 1876
Processing: 1877
Processing: 1878
Processing: 1879
Processing: 1880
Processing: 1881
Processing: 1882
Processing: 1883
Processing: 1884
Processing: 1885
Processing: 1886
Processing: 1887
Processing: 1888
Processing: 1889
Processing: 1890
Processing: 1891
Processing: 1892
Processing: 1893
Processing: 1894
Processing: 1895
Processing: 1896
Processing: 1897
Processing: 1898
Processing: 1899
Processing: 1900
saved
Processing: 1901
Processing: 1902
Processing: 1903
Processing: 1904
Processing: 1905
Processing: 1906
Processing: 1907
Processing: 1908
Processi

Processing: 2330
Processing: 2331
Processing: 2332
Processing: 2333
Processing: 2334
Processing: 2335
Processing: 2336
Processing: 2337
Processing: 2338
Processing: 2339
Processing: 2340
Processing: 2341
Processing: 2342
Processing: 2343
Processing: 2344
Processing: 2345
Processing: 2346
Processing: 2347
Processing: 2348
Processing: 2349
Processing: 2350
saved
Processing: 2351
Processing: 2352
Processing: 2353
Processing: 2354
Processing: 2355
Processing: 2356
Processing: 2357
Processing: 2358
Processing: 2359
Processing: 2360
Processing: 2361
Processing: 2362
Processing: 2363
Processing: 2364
Processing: 2365
Processing: 2366
Processing: 2367
Processing: 2368
Processing: 2369
Processing: 2370
Processing: 2371
Processing: 2372
Processing: 2373
Processing: 2374
Processing: 2375
Processing: 2376
Processing: 2377
Processing: 2378
Processing: 2379
Processing: 2380
Processing: 2381
Processing: 2382
Processing: 2383
Processing: 2384
Processing: 2385
Processing: 2386
Processing: 2387
Processi

Processing: 2809
Processing: 2810
Processing: 2811
Processing: 2812
Processing: 2813
Processing: 2814
Processing: 2815
Processing: 2816
Processing: 2817
Processing: 2818
Processing: 2819
Processing: 2820
Processing: 2821
Processing: 2822
Processing: 2823
Processing: 2824
Processing: 2825
Processing: 2826
Processing: 2827
Processing: 2828
Processing: 2829
Processing: 2830
Processing: 2831
Processing: 2832
Processing: 2833
Processing: 2834
Processing: 2835
Processing: 2836
Processing: 2837
Processing: 2838
Processing: 2839
Processing: 2840
Processing: 2841
Processing: 2842
Processing: 2843
Processing: 2844
Processing: 2845
Processing: 2846
Processing: 2847
Processing: 2848
Processing: 2849
Processing: 2850
saved
Processing: 2851
Processing: 2852
Processing: 2853
Processing: 2854
Processing: 2855
Processing: 2856
Processing: 2857
Processing: 2858
Processing: 2859
Processing: 2860
Processing: 2861
Processing: 2862
Processing: 2863
Processing: 2864
Processing: 2865
Processing: 2866
Processi

Processing: 3286
Processing: 3287
Processing: 3288
Processing: 3289
Processing: 3290
Processing: 3291
Processing: 3292
Processing: 3293
Processing: 3294
Processing: 3295
Processing: 3296
Processing: 3297
Processing: 3298
Processing: 3299
Processing: 3300
saved
Processing: 3301
Processing: 3302
Processing: 3303
Processing: 3304
Processing: 3305
Processing: 3306
Processing: 3307
Processing: 3308
Processing: 3309
Processing: 3310
Processing: 3311
Processing: 3312
Processing: 3313
Processing: 3314
Processing: 3315
Processing: 3316
Processing: 3317
Processing: 3318
Processing: 3319
Processing: 3320
Processing: 3321
Processing: 3322
Processing: 3323
Processing: 3324
Processing: 3325
Processing: 3326
Processing: 3327
Processing: 3328
Processing: 3329
Processing: 3330
Processing: 3331
Processing: 3332
Processing: 3333
Processing: 3334
Processing: 3335
Processing: 3336
Processing: 3337
Processing: 3338
Processing: 3339
Processing: 3340
Processing: 3341
Processing: 3342
Processing: 3343
Processi

Processing: 3765
Processing: 3766
Processing: 3767
Processing: 3768
Processing: 3769
Processing: 3770
Processing: 3771
Processing: 3772
Processing: 3773
Processing: 3774
Processing: 3775
Processing: 3776
Processing: 3777
Processing: 3778
Processing: 3779
Processing: 3780
Processing: 3781
Processing: 3782
Processing: 3783
Processing: 3784
Processing: 3785
Processing: 3786
Processing: 3787
Processing: 3788
Processing: 3789
Processing: 3790
Processing: 3791
Processing: 3792
Processing: 3793
Processing: 3794
Processing: 3795
Processing: 3796
Processing: 3797
Processing: 3798
Processing: 3799
Processing: 3800
saved
Processing: 3801
Processing: 3802
Processing: 3803
Processing: 3804
Processing: 3805
Processing: 3806
Processing: 3807
Processing: 3808
Processing: 3809
Processing: 3810
Processing: 3811
Processing: 3812
Processing: 3813
Processing: 3814
Processing: 3815
Processing: 3816
Processing: 3817
Processing: 3818
Processing: 3819
Processing: 3820
Processing: 3821
Processing: 3822
Processi

-------------------

-------------------
<a id='Incomplete_Data'></a>
### <font color='red'>Problem1:</font>
## At the EDA stage we discovered that the data collection was not carried out properly:
see: [4_EDA](./4_EDA.ipynb)

in the 'href_crawling' file we saved the links of the missing games to mid_games.csv file

see: [href_crawling](./1_href_crawling.ipynb) => <font color='red'>'Problem1: Incomplete data'</font>

now we will save the game data to a CSV file:

## saving the Mid Rated Games, by runing the function to the 'mid_games.csv' file

In [9]:
get_games_data('hrefs_csv/mid_games.csv', 'mid_rated_games')

Processing: 0
Processing: 1
Processing: 2
Processing: 3
Processing: 4
Processing: 5
Processing: 6
Processing: 7
Processing: 8
Processing: 9
Processing: 10
Processing: 11
Processing: 12
Processing: 13
Processing: 14
Processing: 15
Processing: 16
Processing: 17
Processing: 18
Processing: 19
Processing: 20
Processing: 21
Processing: 22
Processing: 23
Processing: 24
Processing: 25
Processing: 26
Processing: 27
Processing: 28
Processing: 29
Processing: 30
Processing: 31
Processing: 32
Processing: 33
Processing: 34
Processing: 35
Processing: 36
Processing: 37
Processing: 38
Processing: 39
Processing: 40
Processing: 41
Processing: 42
Processing: 43
Processing: 44
Processing: 45
Processing: 46
Processing: 47
Processing: 48
Processing: 49
Processing: 50
saved
Processing: 51
Processing: 52
Processing: 53
Processing: 54
Processing: 55
Processing: 56
Processing: 57
Processing: 58
Processing: 59
Processing: 60
Processing: 61
Processing: 62
Processing: 63
Processing: 64
Processing: 65
Processing: 66

Processing: 516
Processing: 517
Processing: 518
Processing: 519
Processing: 520
Processing: 521
Processing: 522
Processing: 523
Processing: 524
Processing: 525
Processing: 526
Processing: 527
Processing: 528
Processing: 529
Processing: 530
Processing: 531
Processing: 532
Processing: 533
Processing: 534
Processing: 535
Processing: 536
Processing: 537
Processing: 538
Processing: 539
Processing: 540
Processing: 541
Processing: 542
Processing: 543
Processing: 544
Processing: 545
Processing: 546
Processing: 547
Processing: 548
Processing: 549
Processing: 550
saved
Processing: 551
Processing: 552
Processing: 553
Processing: 554
Processing: 555
Processing: 556
Processing: 557
Processing: 558
Processing: 559
Processing: 560
Processing: 561
Processing: 562
Processing: 563
Processing: 564
Processing: 565
Processing: 566
Processing: 567
Processing: 568
Processing: 569
Processing: 570
Processing: 571
Processing: 572
Processing: 573
Processing: 574
Processing: 575
Processing: 576
Processing: 577
Pr

Processing: 1023
Processing: 1024
Processing: 1025
Processing: 1026
Processing: 1027
Processing: 1028
Processing: 1029
Processing: 1030
Processing: 1031
Processing: 1032
Processing: 1033
Processing: 1034
Processing: 1035
Processing: 1036
Processing: 1037
Processing: 1038
Processing: 1039
Processing: 1040
Processing: 1041
Processing: 1042
Processing: 1043
Processing: 1044
Processing: 1045
Processing: 1046
Processing: 1047
Processing: 1048
Processing: 1049
Processing: 1050
saved
Processing: 1051
Processing: 1052
Processing: 1053
Processing: 1054
Processing: 1055
Processing: 1056
Processing: 1057
Processing: 1058
Processing: 1059
Processing: 1060
Processing: 1061
Processing: 1062
Processing: 1063
Processing: 1064
Processing: 1065
Processing: 1066
Processing: 1067
Processing: 1068
Processing: 1069
Processing: 1070
Processing: 1071
Processing: 1072
Processing: 1073
Processing: 1074
Processing: 1075
Processing: 1076
Processing: 1077
Processing: 1078
Processing: 1079
Processing: 1080
Processi

Processing: 1502
Processing: 1503
Processing: 1504
Processing: 1505
Processing: 1506
Processing: 1507
Processing: 1508
Processing: 1509
Processing: 1510
Processing: 1511
Processing: 1512
Processing: 1513
Processing: 1514
Processing: 1515
Processing: 1516
Processing: 1517
Processing: 1518
Processing: 1519
Processing: 1520
Processing: 1521
Processing: 1522
Processing: 1523
Processing: 1524
Processing: 1525
Processing: 1526
Processing: 1527
Processing: 1528
Processing: 1529
Processing: 1530
Processing: 1531
Processing: 1532
Processing: 1533
Processing: 1534
Processing: 1535
Processing: 1536
Processing: 1537
Processing: 1538
Processing: 1539
Processing: 1540
Processing: 1541
Processing: 1542
Processing: 1543
Processing: 1544
Processing: 1545
Processing: 1546
Processing: 1547
Processing: 1548
Processing: 1549
Processing: 1550
saved
Processing: 1551
Processing: 1552
Processing: 1553
Processing: 1554
Processing: 1555
Processing: 1556
Processing: 1557
Processing: 1558
Processing: 1559
Processi

Status code: 500
Status code mobile: 500
Processing: 1976
Status code: 500
Status code mobile: 500
Processing: 1977
Processing: 1978
Processing: 1979
Processing: 1980
Processing: 1981
Processing: 1982
Processing: 1983
Processing: 1984
Processing: 1985
Processing: 1986
Processing: 1987
Processing: 1988
Processing: 1989
Processing: 1990
Processing: 1991
Processing: 1992
Processing: 1993
Processing: 1994
Processing: 1995
Processing: 1996
Processing: 1997
Processing: 1998
Processing: 1999
Processing: 2000
saved
Processing: 2001
Processing: 2002
Processing: 2003
Processing: 2004
Processing: 2005
Processing: 2006
Processing: 2007
Processing: 2008
Processing: 2009
Processing: 2010
Processing: 2011
Processing: 2012
Processing: 2013
Processing: 2014
Processing: 2015
Processing: 2016
Processing: 2017
Processing: 2018
Processing: 2019
Processing: 2020
Processing: 2021
Processing: 2022
Processing: 2023
Processing: 2024
Processing: 2025
Processing: 2026
Processing: 2027
Processing: 2028
Processing:

...

Processing: 4843
Processing: 4844
Processing: 4845
Processing: 4846
Processing: 4847
Processing: 4848
Processing: 4849
Processing: 4850
saved
Processing: 4851
Processing: 4852
Processing: 4853
Processing: 4854
Processing: 4855
Processing: 4856
Processing: 4857
Processing: 4858
Processing: 4859
Processing: 4860
Processing: 4861
Processing: 4862
Processing: 4863
Processing: 4864
Processing: 4865
Processing: 4866
Processing: 4867
Processing: 4868
Processing: 4869
Processing: 4870
Processing: 4871
Processing: 4872
Processing: 4873
Processing: 4874
Processing: 4875
Processing: 4876
Processing: 4877
Processing: 4878
Processing: 4879
Processing: 4880
Processing: 4881
Processing: 4882
Processing: 4883
Processing: 4884
Processing: 4885
Processing: 4886
Processing: 4887
Processing: 4888
Processing: 4889
Processing: 4890
Processing: 4891
Processing: 4892
Processing: 4893
Processing: 4894
Processing: 4895
Processing: 4896
Processing: 4897
Processing: 4898
Processing: 4899
Processing: 4900
saved
Pr

Processing: 5322
Processing: 5323
Processing: 5324
Processing: 5325
Processing: 5326
Processing: 5327
Processing: 5328
Processing: 5329
Processing: 5330
Processing: 5331
Processing: 5332
Processing: 5333
Processing: 5334
Processing: 5335
Processing: 5336
Processing: 5337
Processing: 5338
Processing: 5339
Processing: 5340
Processing: 5341
Processing: 5342
Processing: 5343
Processing: 5344
Processing: 5345
Processing: 5346
Processing: 5347
Processing: 5348
Processing: 5349
Processing: 5350
saved
Processing: 5351
Processing: 5352
Processing: 5353
Processing: 5354
Processing: 5355
Processing: 5356
Processing: 5357
Processing: 5358
Processing: 5359
Processing: 5360
Processing: 5361
Processing: 5362
Processing: 5363
Processing: 5364
Processing: 5365
Processing: 5366
Processing: 5367
Processing: 5368
Processing: 5369
Processing: 5370
Processing: 5371
Processing: 5372
Processing: 5373
Processing: 5374
Processing: 5375
Processing: 5376
Processing: 5377
Processing: 5378
Processing: 5379
Processi

Processing: 5796
Processing: 5797
Processing: 5798
Processing: 5799
Processing: 5800
saved
Processing: 5801
Processing: 5802
Processing: 5803
Processing: 5804
Processing: 5805
Processing: 5806
Processing: 5807
Processing: 5808
Processing: 5809
Processing: 5810
Processing: 5811
Processing: 5812
Processing: 5813
Processing: 5814
Processing: 5815
Processing: 5816
Processing: 5817
Processing: 5818
Processing: 5819
Processing: 5820
Processing: 5821
Processing: 5822
Processing: 5823
Processing: 5824
Processing: 5825
Processing: 5826
Processing: 5827
Processing: 5828
Processing: 5829
Processing: 5830
Processing: 5831
Processing: 5832
Processing: 5833
Processing: 5834
Processing: 5835
Processing: 5836
Processing: 5837
Processing: 5838
Processing: 5839
Processing: 5840
Processing: 5841
Processing: 5842
Processing: 5843
Processing: 5844
Processing: 5845
Processing: 5846
Processing: 5847
Processing: 5848
Processing: 5849
Processing: 5850
saved
Processing: 5851
Processing: 5852
Processing: 5853
Pr

Processing: 6275
Processing: 6276
Processing: 6277
Processing: 6278
Processing: 6279
Processing: 6280
Processing: 6281
Processing: 6282
Processing: 6283
Processing: 6284
Processing: 6285
Processing: 6286
Processing: 6287
Processing: 6288
Processing: 6289
Processing: 6290
Processing: 6291
Processing: 6292
Processing: 6293
Processing: 6294
Processing: 6295
Processing: 6296
Processing: 6297
Processing: 6298
Processing: 6299
Processing: 6300
saved
Processing: 6301
Processing: 6302
Processing: 6303
Processing: 6304
Processing: 6305
Processing: 6306
Processing: 6307
Processing: 6308
Processing: 6309
Processing: 6310
Processing: 6311
Processing: 6312
Processing: 6313
Processing: 6314
Processing: 6315
Processing: 6316
Processing: 6317
Processing: 6318
Processing: 6319
Processing: 6320
Processing: 6321
Processing: 6322
Processing: 6323
Processing: 6324
Processing: 6325
Processing: 6326
Processing: 6327
Processing: 6328
Processing: 6329
Processing: 6330
Processing: 6331
Processing: 6332
Processi

Processing: 6754
Processing: 6755
Processing: 6756
Processing: 6757
Processing: 6758
Processing: 6759
Processing: 6760
Processing: 6761
Processing: 6762
Processing: 6763
Processing: 6764
Processing: 6765
Processing: 6766
Processing: 6767
Processing: 6768
Processing: 6769
Processing: 6770
Processing: 6771
Processing: 6772
Processing: 6773
Processing: 6774
Processing: 6775
Processing: 6776
Processing: 6777
Processing: 6778
Processing: 6779
Processing: 6780
Processing: 6781
Processing: 6782
Processing: 6783
Processing: 6784
Processing: 6785
Processing: 6786
Processing: 6787
Processing: 6788
Processing: 6789
Processing: 6790
Processing: 6791
Processing: 6792
Processing: 6793
Processing: 6794
Processing: 6795
Processing: 6796
Processing: 6797
Processing: 6798
Processing: 6799
Processing: 6800
saved
Processing: 6801
Processing: 6802
Processing: 6803
Processing: 6804
Processing: 6805
Processing: 6806
Processing: 6807
Processing: 6808
Processing: 6809
Processing: 6810
Processing: 6811
Processi

### We will continue the problem handling in the 'data_handling' file:

see: [data_handling](./3_data_handling.ipynb) => <font color='red'>'Problem1: Incomplete data'</font>

-------------------